In [2]:
# imports

import pandas as pd
import requests
from dotenv import load_dotenv
import os
import numpy as np
import json




# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Access API Keys
load_dotenv(dotenv_path='/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data/.env')
api_key = os.getenv('FOURSQUARE_Project_API_KEY')


ZC4VKAL4HIK31TYKWXT00NQOCOYF23ZTUYD1IB3OZ2QYBE01


In [11]:
# Upload Lisbon CityBike Data .CSV
lb_citybikes = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data/lisbon_bike_station_df.csv')

lb_citybikes

,latitude,longitude,timestamp,free bikes,empty slots,total slots
0,38.753590,-9.157170,2025-07-25T01:06:43.220041+00:00Z,1,31,32
1,38.752310,-9.158850,2025-07-25T01:06:43.219966+00:00Z,0,23,23
2,38.770657,-9.160248,2025-07-25T01:06:43.219971+00:00Z,5,28,33
3,38.780690,-9.096220,2025-07-25T01:06:43.220291+00:00Z,10,4,14
4,38.702936,-9.175234,2025-07-25T01:06:43.220528+00:00Z,8,15,23
...,...,...,...,...,...,...
190,38.729724,-9.157531,2025-07-25T01:06:43.220410+00:00Z,4,26,30
191,38.711498,-9.194361,2025-07-25T01:06:43.220763+00:00Z,2,39,41
192,38.739395,-9.162447,2025-07-25T01:06:43.220169+00:00Z,0,17,17
193,38.708780,-9.137125,2025-07-25T01:06:43.220502+00:00Z,13,7,20


In [42]:
# Parameters
lb_citybikes['coordinates'] = lb_citybikes['latitude'].astype(str) + ',' + lb_citybikes['longitude'].astype(str)
lb_citybikes['station_id'] = lb_citybikes.index

lb_citybikes.head(5)

,latitude,longitude,timestamp,free bikes,empty slots,total slots,coordinates,station_id
0,38.753590,-9.157170,2025-07-25T01:06:43.220041+00:00Z,1,31,32,"38.75359,-9.15717",0
1,38.752310,-9.158850,2025-07-25T01:06:43.219966+00:00Z,0,23,23,"38.75231,-9.15885",1
2,38.770657,-9.160248,2025-07-25T01:06:43.219971+00:00Z,5,28,33,"38.770657,-9.160248",2
3,38.780690,-9.096220,2025-07-25T01:06:43.220291+00:00Z,10,4,14,"38.78069,-9.09622",3
4,38.702936,-9.175234,2025-07-25T01:06:43.220528+00:00Z,8,15,23,"38.702936,-9.175234",4


In [ ]:
# Create Parameters
ll = lb_citybikes['coordinates']

# Assign Parameters
paramters = {
    'll' : ll,
    'radius':1000,
    'fields':'name,fsq_place_id,distance,categories,attributes,price,popularity,rating,stats,hours,hours_popular,veracity_rating',
}
# Assign API URL
url = 'https://places-api.foursquare.com/places/search'

# Assign required headers
headers = {
    'accept': 'application/json',
    'authorization': 'Bearer'+api_key,
    'X-Places-Api-Version': '2025-06-17'
}

ll


0        38.75359,-9.15717
1        38.75231,-9.15885
2      38.770657,-9.160248
3        38.78069,-9.09622
4      38.702936,-9.175234
              ...         
190    38.729724,-9.157531
191    38.711498,-9.194361
192    38.739395,-9.162447
193     38.70878,-9.137125
194      38.74095,-9.13409
Name: coordinates, Length: 195, dtype: object

In [ ]:
# FOURSQUARE Places API
responses = []

# Iterate through each station by its coordinates
for coordinates in ll:

    paramters = {
    'll' : coordinates,
    'radius':1000,
    'fields':'name,fsq_place_id,distance,categories,attributes,price,popularity,rating,stats,hours,hours_popular,veracity_rating',
    }
    
    response = requests.get(url,headers=headers,params=paramters)
    data = response.json()
    responses.append(data)

In [ ]:
import pprint

# View FOURSQUARE API Responses
pprint.pprint(responses)


[{'context': {'geo_bounds': {'circle': {'center': {'latitude': 38.75359,
                                                   'longitude': -9.15717},
                                        'radius': 1000}}},
  'results': [{'attributes': {},
               'categories': [{'fsq_category_id': '4bf58dd8d48988d126941735',
                               'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
                                        'suffix': '.png'},
                               'name': 'Government Building',
                               'plural_name': 'Government Buildings',
                               'short_name': 'Government'},
                              {'fsq_category_id': '4bf58dd8d48988d12f941735',
                               'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
                                        'suffix': '.png'},
                               'name': 'Library',
                     

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Create empty lists 
popularity_list = []
rating_list = []
total_rating_list = []
veracity_rating_list = []
distance_list = []
name_list = []
category_name_list = []
hours_list = []
coordinates_list = []

# Iterate through each station_coordinates results
for stations in responses:
    context = stations.get('context', {})
    center = context.get('geo_bounds', {}).get('circle', {}).get('center', {})
    latitute = center.get('latitude', 'N/A')
    longitude = center.get('longitude', 'N/A')
    
    results = stations.get('results','N/A')
    # Iterate through each station's results for data on Points_Of_Interest
    for station in results:
        name = station.get('name','N/A')
        rating = station.get('rating','N/A')
        popularity = station.get('popularity','N/A')
        distance = station.get('distance','N/A')
        veracity_rating = station.get('veracity_rating','N/A')

        stats = station.get('stats',{})
        total_ratings = stats.get('total_ratings',{})

        categories = station.get('categories', [])
        category_name = ', '.join([cat.get('name', 'N/A') for cat in categories])

        # Append each Point_Of_Interest to empty lists
        popularity_list.append(popularity)
        rating_list.append(popularity)
        total_rating_list.append(total_ratings)
        veracity_rating_list.append(veracity_rating)
        distance_list.append(distance)
        name_list.append(name)
        category_name_list.append(category_name)
        coordinates_list.append(f"{latitute},{longitude}")

        


    



In [ ]:
# Create dictionary of iterated list entires
station_POI_dic = {
    'name':name_list,
    'rating':rating_list,
    'total num ratings':total_rating_list,
    'popularity':popularity_list,
    'distance':distance_list,
    'veracity':veracity_rating_list,
    'category': category_name_list,
    'coordinates': coordinates_list

}

# Create DataFrame from Dictionary
station_POI_df = pd.DataFrame(station_POI_dic)

station_POI_df.head(5)

,name,rating,total num ratings,popularity,distance,veracity,category,coordinates
0,Torre Do Tombo,0.944024,21,0.944024,124,4,"Government Building, Library","38.75359,-9.15717"
1,Aula Magna,0.987013,127,0.987013,138,4,Concert Hall,"38.75359,-9.15717"
2,Alameda da Universidade,0.938399,47,0.938399,121,4,"Plaza, Garden","38.75359,-9.15717"
3,Letras Bar,0.781422,11,0.781422,87,3,Portuguese Restaurant,"38.75359,-9.15717"
4,Horto do Campo Grande,0.959710,40,0.959710,337,5,Flower Store,"38.75359,-9.15717"


Put your parsed results into a DataFrame

In [ ]:
# Assess Datframe Success
station_POI_df['coordinates'].value_counts()

coordinates
38.75359,-9.15717      10
38.75264,-9.15671      10
38.771913,-9.11314     10
38.758691,-9.19216     10
38.760986,-9.18327     10
                       ..
38.73434,-9.14446      10
38.770808,-9.121902    10
38.724954,-9.149334    10
38.777548,-9.094686    10
38.74095,-9.13409      10
Name: count, Length: 195, dtype: int64

In [ ]:
# Assign station_id in station_POI dataframe
station_POI_df = station_POI_df.merge(
    lb_citybikes[['coordinates', 'station_id']],
    on='coordinates',
    how='left'
)

# Remove station coorindates to minimze data redunancy and increase readability

station_POI_df.drop(columns=['coordinates'], inplace=True)




KeyError: 'coordinates'

In [ ]:
# Download CSV File
station_POI_df.to_csv('FOURSQUARE_Lisbon_Station_Surroundings.csv', index=False, header=True, sep=',')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [1]:
# Yelp](https://docs.developer.yelp.com/docs/fusion-intro) 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating